# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## Lesson

Anthropic offers two APIs, the legacy [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) and the current [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html). For this tutorial, we will be exclusively using the Messages API.

At minimum, a call to Claude using the Messages API requires the following parameters:
- `model`: the [API model name](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns) of the model that you intend to call

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

- `messages`: an array of input messages. Our models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the user `role`.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit our [API documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [2]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [3]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown depending on the seafloor.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect on the surface of the ocean, making it appear more gray, white, or even black on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the water and the lack of suspended particles in the deeper parts of the ocean. But the ocean's color can definitely vary quite a bit depending 

In [4]:
# Prompt
PROMPT = "What year was Donald trump born in?"

# Print Claude's response
print(get_completion(PROMPT))

Donald Trump was born in 1946. Specifically, he was born on June 14, 1946 in Queens, New York.


Now let's take a look at some prompts that do not include the correct Messages API formatting. For these malformatted prompts, the Messages API returns an error.

First, we have an example of a Messages API call that lacks `role` and `content` fields in the `messages` array.

> ⚠️ **Warning:** Due to the incorrect formatting of the messages parameter in the prompt, the following cell will return an error. This is expected behavior.

In [5]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{ "role": "user", "content": "Hi Claude, how are you?"} ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

response_body = json.loads(response["body"].read())
print(response_body["content"][0]["text"])

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


Here's a prompt that fails to alternate between the `user` and `assistant` roles.

> ⚠️ **Warning:** Due to the lack of alternation between `user` and `assistant` roles, Claude will return an error message. This is expected behavior.

In [6]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: messages: roles must alternate between "user" and "assistant", but found multiple "user" roles in a row

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What is the composition of the Earth's atmosphere, and how does that affect the way light interacts with it?
- How do different wavelengths of light behave as they pass through the atmosphere?
- What is the process known as Rayleigh scattering, and how does it contribute to the blue appearance of the sky?
- How do factors like the time of day, weather conditions, and location on Earth influence the appearance of the sky?
- What other celestial bodies in our solar system have differently colored skies, and what might explain those differences?
- How have our scientific understanding and theories about the sky's color evolved over time as we've gained new knowledge?
- What are some of the practical applications of understanding why the sky appears blue?

The goal of these questions is to prompt further inquiry and critical analysis, rather than providing a direct answer. Let me know if you have any other

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[could you count 1 to 3 ?]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1, 2, 3.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Behave like a 3 year old child]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

*looks up at the sky with wide eyes* Wow, the sky is so big! It goes on forever and ever! I can't even see the end of it! *giggles* It's so blue and pretty today. I wanna fly up there with the birdies! *reaches up towards the sky* Can we go play in the clouds? Please please please?

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_1_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown depending on the seafloor.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect on the surface of the ocean, making it appear more gray, white, or even black on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the water and the lack of suspended particles in the deeper parts of the ocean. But the ocean's color can definitely vary quite a bit depending 

In [ ]:
# Prompt
PROMPT = "What year was Donald trump born in?"

# Print Claude's response
print(get_completion(PROMPT))

Donald Trump was born on June 14, 1946.


In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"Hi Claude, how are you?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

TypeError: Object of type set is not JSON serializable

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: messages: roles must alternate between "user" and "assistant", but found multiple "user" roles in a row

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What causes the sky to appear blue in color?
- How does the composition of the Earth's atmosphere contribute to the sky's blue appearance?
- What role do the wavelengths of visible light play in making the sky appear blue?
- How do the scattering properties of different wavelengths of light affect the sky's color?
- What other factors, such as the time of day or weather conditions, can influence the sky's appearance?
- How have scientists historically explained and studied the phenomenon of the blue sky?
- What are some alternative perspectives or theories that have been proposed to explain the sky's blue color?

By exploring these types of questions, we can delve deeper into the scientific principles and mechanisms that give rise to the familiar blue hue of the sky.
